In [1]:
import os
import pandas as pd

In [2]:
# read survey data and standard variable dictionary
survey_raw = pd.read_csv('test_actransit.csv')
var_dict = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\onboard-surveys\util\dict_v1.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (119,120) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# check no duplicated records and no duplicated id

survey_nodup = survey_raw.drop_duplicates()
assert survey_nodup.shape[0] == survey_raw.shape[0], 'Survey data has duplicated records.'

assert ('ID' in survey_nodup), "Survey data is missing the 'ID' field."

assert survey_nodup.shape[0] == len(survey_nodup.ID.unique()), 'ID is not identical.'

print('Survey data contains {} records.'.format(survey_nodup.shape[0]))
df = survey_nodup.copy()

Survey data contains 14678 records.


In [4]:
# check if the data contains all the required fields

var_all = var_dict.generic_variable.unique()
for i in var_all:
    assert (i in list(df)), 'Survey data is missing field: {}'.format(i)

print('Survey data has all required fields.')

Survey data has all required fields.


In [5]:
# check if categoric variables have the valid values or too many missing data

missing_pct = 0.1

var_cat = var_dict.loc[var_dict.variable_type == 'categoric']['generic_variable'].unique()

for i in var_cat:
    #print(i)
    df_sub = df[['ID', i]]
    var_sub = var_dict.loc[var_dict.generic_variable == i][['generic_variable','valid_values_for_categoric_variables']]
    
    compare = df_sub.merge(var_sub, left_on=i, right_on='valid_values_for_categoric_variables', how='left')
    #display(compare)
    diff = compare.loc[compare.generic_variable.isnull()]
    if diff.shape[0] > 0:
 
        df_na = diff.loc[diff[i].isnull()]
        df_na_count = df_na.shape[0]
        if df_na_count > 0:
            print('{} records have missing data for variable "{}"'.format(df_na_count, i))                
            # na_pct = diff_na.shape[0] / df.shape[0]
            # assert (na_pct <= missing_pct), 'Variable {} has too many missing data'.format(i)

        df_non_na = diff.loc[diff[i].notnull()]
        df_other_value = df_non_na[i].unique()
        if df_non_na.shape[0] > 0:
            print('Variable "{}" has the following values that are not in the standard dictionary :\n{}'.format(i, df_other_value))


21 records have missing data for variable "orig_purp"
7 records have missing data for variable "egress_mode"
13 records have missing data for variable "dest_purp"
5479 records have missing data for variable "at_work_prior_to_orig_purp"
5479 records have missing data for variable "at_work_after_dest_purp"
10701 records have missing data for variable "at_school_prior_to_orig_purp"
10701 records have missing data for variable "at_school_after_dest_purp"
Variable "fare_category" has the following values that are not in the standard dictionary :
['adult' 'youth' 'disabled' 'senior']
Variable "fare_medium" has the following values that are not in the standard dictionary :
['cash' 'clipper']
117 records have missing data for variable "gender"
3319 records have missing data for variable "household_income"
10861 records have missing data for variable "eng_proficient"


In [6]:
# for non-categoric variables, check #missing data

## variables that should have no missing data:

no_na_variables =  ['time_string',
                    'date_string',
                    'route',
                    'survey_alight_lat',
                    'survey_alight_lon',
                    'survey_board_lat',
                    'survey_board_lon',
                    'orig_purp',
                    'orig_lat',
                    'orig_lon',
                    'dest_purp',
                    'dest_lat',
                    'dest_lon',
                    'home_lat',
                    'home_lon']
df_chk_na = df[no_na_variables]

df_no_na = df_chk_na.dropna(how='any')
df_na_pct = 1 - df_no_na.shape[0]/df.shape[0]

print('{} records are missing data in at least one of the following variables: \n{}'.format(df.shape[0]-df_no_na.shape[0],
                                                                                            no_na_variables))

# assert (df_na_pct <= missing_pct), '{} records ({} of all records) are missing data \
#                                     in the following variables, \
#                                     exceeding the threshold {}: \n{}'.format(df.shape[0]-df_no_na.shape[0],
#                                                                              df_na_pct,
#                                                                              missing_pct,
#                                                                              no_na_variables)


134 records are missing data in at least one of the following variables: 
['time_string', 'date_string', 'route', 'survey_alight_lat', 'survey_alight_lon', 'survey_board_lat', 'survey_board_lon', 'orig_purp', 'orig_lat', 'orig_lon', 'dest_purp', 'dest_lat', 'dest_lon', 'home_lat', 'home_lon']


In [7]:
## workers should have workplace_lat/_lon; students should have school_lat/_lon
workers = df.loc[df.work_status == 'full- or part-time']
workers_lat_lon = workers.loc[(workers.workplace_lat.notnull()) & (workers.workplace_lon.notnull())]
workers_missing_latlon = workers.shape[0] - workers_lat_lon.shape[0]
if workers_missing_latlon > 0:
    print('{} workers missing workplace_lat/lon'.format(workers_missing_latlon))
# assert (workers.shape[0] == workers_lat_lon.shape[0]), 'workers missing workplace_lat/_lon'

students = df.loc[df.student_status == 'full- or part-time']
students_lat_lon = students.loc[(students.school_lat.notnull()) & (students.school_lon.notnull())]
students_missing_latlon = students.shape[0] - students_lat_lon.shape[0]
if students_missing_latlon > 0:
    print('{} students missing school_lat/lon'.format(students_missing_latlon))
# assert (students.shape[0] == students_lat_lon.shape[0]), 'students missing school_lat/_lon'

171 students missing school_lat/lon


In [8]:
# check transfer routes should have both operator and route number/name info?
# access_mode / egress_mode more categories?
# missing values - "missing" or NA?